# Videojuegos exitosos

## Índice <a id='back'></a>
* [Introducción](#intro)
* [Etapa 1. Descripción y preprocesamiento de los datos](#data_review)
    * [1.1. Duplicados](#data_review_duplicated)
        * [1.1.1. Columna `'platform'`](#data_review_duplicated_platform)
        * [1.1.2. Columna `'genre'`](#data_review_duplicated_genre)
        * [1.1.3. Columna `'user_score'`](#data_review_duplicated_userscore)
        * [1.1.4. Columna `'rating'`](#data_review_duplicated_rating)
        * [1.1.5. Columna `'name'`](#data_review_duplicated_name)
    * [1.2. Valores ausentes](#data_review_missing)
        * [1.2.1. Columna `'year_of_release'`](#data_review_missing_year)
        * [1.2.2. Columna `'critic_score'`](#data_review_missing_criticscore)
        * [1.2.3. Columna `'user_score'`](#data_review_missing_userscore)
    * [1.3. Conclusiones](#data_review_conclusions)
* [Etapa 2. Preparación de los datos](#data_prep)
    * [2.1. Tarifa por usuario](#data_prep_income)
* [Etapa 3. Análisis exploratorio de datos](#data_analysis)
    * [3.1. Llamadas](#data_analysis_calls)
    * [3.2. Mensajes](#data_analysis_messages)
    * [3.3. Internet](#data_analysis_mb)
    * [3.4. Ingreso](#data_analysis_income)
    * [3.4. Conclusiones](#data_analysis_conclusions)
* [Etapa 4. Comprobación de hipótesis](#data_hypothesis)
    * [4.1. El ingreso difiere por plan](#data_hypothesis_plan)
    * [4.2. El ingreso difiere por región](#data_hypothesis_zone)
* [Etapa 5. Conclusión general](#data_conclusion)

## Introducción <a id='intro'></a>

Este proyecto es realizado para la tienda online Ice, la cual vende videojuegos por todo el mundo. Se cuenta con datos del año 2016.

**Objetivo e Hipótesis**

El objetivo es identificar patrones para determinar si un juego tiene éxito o no. Se establecen dos hipótesis:
1. Las puntuaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas.
2. Las puntuaciones promedio de los usuarios para los géneros de Acción y Deportes son distintas.

**Etapas**
Los datos se almacenan en el archivo `/datasets/games.csv`. No hay ninguna información sobre la calidad de los datos así que se examinarán antes de probar las hipótesis.

Primero, se evaluará la calidad de los datos y si los problemas son significativos, entonces, durante el preprocesamiento de datos, se tomará en cuenta los problemas más críticos. Finalmente se comprobarán las hipótesis para presentar las conclusiones.
 
El proyecto consistirá en cinco etapas:
 1. Descripción y preprocesamiento de los datos.
 2. Preparación de los datos.
 3. Análisis exploratorio de los datos.
 4. Comprobación de hipótesis.
 5. Conclusión general.

[Volver a Contenidos](#back)

## Etapa 1. Descripción y preprocesamiento de los datos <a id='data_review'></a>

Se importan las librerías necesarias.

In [2]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from scipy import stats as st
from math import ceil, factorial

Se lee el archivo y se guarda en la variable `df`.

In [3]:
try:
    df = pd.read_csv('games.csv')
except:
    df = pd.read_csv('/datasets/games.csv')

Se imprime la información general y las primeras 10 filas.

In [4]:
df.info()
print()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB

                        Name Platform  Year_of_Release         Genre  \
0                 Wii Sports      Wii           2006.0        Sports   
1          Super Mario Bros.      NES           1985.0      Platform   
2             Mario Kart

La tabla cuenta con 16,715 filas y 11 columnas, las cuales son:

1. `'Name'` - Título del videojuego.
2. `'Platform'` - Consola o pltaforma del videojuego.
3. `'Year_of_Release'` - Año de lanzamiento del videojuego.
4. `'Genre'` - Género del videojuego.
5. `'NA_sales'` - ventas en Norteamérica, en millones de dólares estadounidenses.
6. `'EU_sales'` - ventas en Europa, en millones de dólares estadounidenses.
7. `'JP_sales'` - ventas en Japón, en millones de dólares estadounidenses.
8. `'Other_sales'` - ventas en otras regiones, en millones de dólares estadounidenses.
9. `'Critic_Score'` - puntuación según la crítica (máximo 100).
10. `'User_Score'` - puntuación según los usuarios (máximo 10).
11. `'Rating'` - clasificación ESRB.

Existen valores ausentes en las columnas `'Name'`, `'Year_of_Release'`, `'genre'`, `'Critic_Score'`, `'User_Score'`y `'Rating'`. Se investigará más sobre cada una de ellas. Los nombres de las columnas usan letras mayúsculas y minúsculas, se cambiarán por minúsculas todas las letras.

Se imprimirán los títulos de las columnas en busca de problemas más sútiles.

In [5]:
print(df.columns)

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'NA_sales', 'EU_sales',
       'JP_sales', 'Other_sales', 'Critic_Score', 'User_Score', 'Rating'],
      dtype='object')


El único detalle es el uso de mayúsculas y minúsculas. Se cambian a minúsculas todas las letras.

In [6]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


[Volver a Contenidos](#back)

### 1. 1. Duplicados <a id='data_review_duplicated'></a>

En esta sección se tratarán las columnas de tipo object, principalmente para analizar si existen duplicados.

#### 1.1.1 Columna `'platform'` <a id='data_review_duplicated_platform'></a>

La columna `'platform'` no contiene valores ausentes. Se imprimirá la lista de valores únicos para determinar si existen duplicados implícitos.

In [7]:
df['platform'] = df['platform'].str.lower() #convertimos a minúsculas los datos de la columna
print(df['platform'].sort_values().unique())

['2600' '3do' '3ds' 'dc' 'ds' 'gb' 'gba' 'gc' 'gen' 'gg' 'n64' 'nes' 'ng'
 'pc' 'pcfx' 'ps' 'ps2' 'ps3' 'ps4' 'psp' 'psv' 'sat' 'scd' 'snes' 'tg16'
 'wii' 'wiiu' 'ws' 'x360' 'xb' 'xone']


Se tienen las siguientes consolas:
* `'2600'` - Atari 2600.
* `'3do'` - 3DO Interactive Multiplayer.
* `'3ds'` - Nintendo 3DS.
* `'dc'` - Sega Dreamcast.
* `'ds'` - Nintendo DS.
* `'gb'` - Nintendo Game Boy.
* `'gba'` - Nintendo Game Boy Advance.
* `'gc'` - Nintendo GameCube.
* `'gen'`- Sega Genesis.
* `'gg'` - Sega Game Gear.
* `'n64'` - Nintendo 64.
* `'nes'` - Nintendo.
* `'ng'` - Neo Geo.
* `'pc'` - Plataforma.
* `'pcfx'` - NEC PC-FX.
* `'ps'` - PlayStation.
* `'ps2'` - PlayStation 2.
* `'ps3'` - PlayStation 3.
* `'ps4'` - PlayStation 4.
* `'psp'` - PlayStation Portable.
* `'psv'` - PlatStation Vita.
* `'sat'` - Sega Saturn.
* `'scd'` - Sega CD.
* `'snes'` - Super Nintendo.
* `'tg16'` - PC Engine.
* `'wii'` - Nintendo Wii.
* `'wiiu'` - Nintendo Wii U.
* `'ws'` - WonderSwan.
* `'x360'` - Xbox 360.
* `'xb'` - Xbox.
* `'xone'` - Xbox One.

No existen duplicados implícitos.

[Volver a Contenidos](#back)

#### 1.1.2 Columna `'genre'` <a id='data_review_duplicated_genre'></a>

La columna `'genre'` contiene dos valores ausentes los cuales pueden deberse a un error en el proceso de recopilación de datos. Se imprimirán estas dos filas para investigación.

In [8]:
print(df[df['genre'].isna()])

      name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659    NaN      gen           1993.0   NaN      1.78      0.53      0.00   
14244  NaN      gen           1993.0   NaN      0.00      0.00      0.03   

       other_sales  critic_score user_score rating  
659           0.08           NaN        NaN    NaN  
14244         0.00           NaN        NaN    NaN  


Estas filas también contienen los dos valores ausentes de la columna `'name'`. Como sólo son 2 filas de 16,715 y además no hay forma de conocer a qué videjuego pertenecen, se decide eliminarlas.

In [9]:
df = df.dropna(subset='genre')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


Se imprimirá la lista de valores únicos de la columna `'genre'`.

In [10]:
df['genre'] = df['genre'].str.lower() #convertimos a minúsculas todos los datos de esta columna
print(df['genre'].sort_values().unique())

['action' 'adventure' 'fighting' 'misc' 'platform' 'puzzle' 'racing'
 'role-playing' 'shooter' 'simulation' 'sports' 'strategy']


No existen duplicados implícitos.

[Volver a Contenidos](#back)

#### 1.1.3 Columna `'user_score'` <a id='data_review_duplicated_userscore'></a>

Esta columna contiene valores ausentes, los cuales serán tratados más adelante. Además, sus datos deberían ser numéricos, sin embargo aparecen como tipo object. Se mostrarán sus valores únicos para investigación. 

In [11]:
print(df['user_score'].sort_values().unique())

['0' '0.2' '0.3' '0.5' '0.6' '0.7' '0.9' '1' '1.1' '1.2' '1.3' '1.4' '1.5'
 '1.6' '1.7' '1.8' '1.9' '2' '2.1' '2.2' '2.3' '2.4' '2.5' '2.6' '2.7'
 '2.8' '2.9' '3' '3.1' '3.2' '3.3' '3.4' '3.5' '3.6' '3.7' '3.8' '3.9' '4'
 '4.1' '4.2' '4.3' '4.4' '4.5' '4.6' '4.7' '4.8' '4.9' '5' '5.1' '5.2'
 '5.3' '5.4' '5.5' '5.6' '5.7' '5.8' '5.9' '6' '6.1' '6.2' '6.3' '6.4'
 '6.5' '6.6' '6.7' '6.8' '6.9' '7' '7.1' '7.2' '7.3' '7.4' '7.5' '7.6'
 '7.7' '7.8' '7.9' '8' '8.1' '8.2' '8.3' '8.4' '8.5' '8.6' '8.7' '8.8'
 '8.9' '9' '9.1' '9.2' '9.3' '9.4' '9.5' '9.6' '9.7' 'tbd' nan]


El único valor que no es numérico es `'tbd'` (to be determined), esto significa que el videojuego aún no ha sido puntuado. Así, este valor es como si se tuviera un valor ausente pues se desconoce la razón por la que el videojuego no ha diso puntuado. Se reemplazará por NaN.

In [12]:
df['user_score'] = df['user_score'].replace('tbd', float('nan'))
df['user_score'] = df['user_score'].astype('float64') #convertimos los datos de la columna tipo float
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB


[Volver a Contenidos](#back)

#### 1.1.4 Columna `'rating'` <a id='data_review_duplicated_rating'></a>

Análogamente a las columnas anteriores, se convertirán las letras de los datos a minúsculas y se imprimirán los valores únicos.

In [13]:
df['rating'] = df['rating'].str.lower()
print(df['rating'].sort_values().unique())

['ao' 'e' 'e10+' 'ec' 'k-a' 'm' 'rp' 't' nan]


Las clasificaciones existentes de acuerdo a The Entertainment Software Rating Board (ESRB) son:

1. `'ao'` - Sólo adultos (mayores de 18 años).
2. `'e'` - Apto para todas las edades.
3. `'e10+'` - Apto para mayores de 10 años.
4. `'ec'` - Apto para niños pequeños.
5. `'k-a'`- Niños y adultos, esta clasificación es la misma que `'e'` pero fue usada antes del año 1998.
6. `'m'` - Apto para mayores de 17 años (la diferencia con `'ao'` es que contienen menos escenas o de menor duración cuyo contenido sea sexual o de violencia).
7. `'rp'` - Indica que la clasificación está pendiente.
8. `'t'` - Apto para mayores de 13 años.

Los valores ausentes se tratarán más adelante. Se reemplazará la clasificación `'k-a'` por `'e'` ya que indican la misma información.

In [14]:
df['rating'] = df['rating'].replace('k-a', 'e')
print(df['rating'].sort_values().unique())
print()
df.info()

['ao' 'e' 'e10+' 'ec' 'm' 'rp' 't' nan]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB


[Volver a Contenidos](#back)

#### 1.1.5 Columna `'name'` <a id='data_review_duplicated_name'></a>

La columna `'name'` no contiene valores ausentes. Puede haber duplicados implícitos por el uso de mayúsculas y minúsculas, o bien, que los títulos comiencen con espacios en blanco. Esto es lo primero que se tratará.

In [15]:
df['name'] = df['name'].str.lower().str.strip()#convertimos a minúsculas los datos y quitamos espacios en blanco al incio o final
print(len(df['name'].sort_values().unique())) #cantidad de valores únicos
print()
print(df.head(10))

11559

                        name platform  year_of_release         genre  \
0                 wii sports      wii           2006.0        sports   
1          super mario bros.      nes           1985.0      platform   
2             mario kart wii      wii           2008.0        racing   
3          wii sports resort      wii           2009.0        sports   
4   pokemon red/pokemon blue       gb           1996.0  role-playing   
5                     tetris       gb           1989.0        puzzle   
6      new super mario bros.       ds           2006.0      platform   
7                   wii play      wii           2006.0          misc   
8  new super mario bros. wii      wii           2009.0      platform   
9                  duck hunt      nes           1984.0       shooter   

   na_sales  eu_sales  jp_sales  other_sales  critic_score  user_score rating  
0     41.36     28.96      3.77         8.45          76.0         8.0      e  
1     29.08      3.58      6.81         

Al observar las filas 6 y 8, se tiene que se trata del mismo título pero para consolas distintas. El título de la fila 8 contiene el nombre de la consola a la que pertenece. Pueden existir más casos como este.

In [16]:
def delete_platform(row):#función para eliminar la plataforma del título si es que coincide con la columna de plataforma
    name = str(row['name'])
    platform = str(row['platform'])
    platform_end = platform.rjust(len(platform)+1) #agregar un espacio en blanco al incio
    platform_start = platform.ljust(len(platform)+1) #agregar un espacio en blanco al final
    if re.search(platform_start, name): #elimina la plataforma si el título comienza con ella
        name = name.strip(platform_start)
        return name
    elif re.search(platform_end, name): #elimina la plataforma si el título termina con ella
        name = name.strip(platform_end)
        return name
    else:
        return name

In [17]:
df_test = pd.DataFrame() #DataFrame de prueba
df_test['name'] = ['sports wii', 'wii-sports', 'g33ds', '3ds fifa']
df_test['platform'] = ['wii', 'wii', '3ds', '3ds']
print(df_test)
print()
df_test['name'] = df_test.apply(delete_platform, axis=1) #se aplica la función al DataFrame de prueba
print(df_test)

         name platform
0  sports wii      wii
1  wii-sports      wii
2       g33ds      3ds
3    3ds fifa      3ds

         name platform
0      sports      wii
1  wii-sports      wii
2       g33ds      3ds
3        fifa      3ds


In [18]:
df['name'] = df.apply(delete_platform, axis=1) #aplicamos la función a df
print(df.head(10))

                       name platform  year_of_release         genre  na_sales  \
0                    sports      wii           2006.0        sports     41.36   
1         super mario bros.      nes           1985.0      platform     29.08   
2                mario kart      wii           2008.0        racing     15.68   
3             sports resort      wii           2009.0        sports     15.61   
4  pokemon red/pokemon blue       gb           1996.0  role-playing     11.27   
5                    tetris       gb           1989.0        puzzle     23.20   
6     new super mario bros.       ds           2006.0      platform     11.28   
7                      play      wii           2006.0          misc     13.96   
8     new super mario bros.      wii           2009.0      platform     14.44   
9                 duck hunt      nes           1984.0       shooter     26.93   

   eu_sales  jp_sales  other_sales  critic_score  user_score rating  
0     28.96      3.77         8.45    

In [19]:
print(df['name'].sort_values().head(20)) #imprimimos los primeros 20 valores de esta columna (ordenados alfabeticamente)

4728                                     '98 koshien
8342                      .hack//g.u. vol.1//rebirth
7087                    .hack//g.u. vol.2//reminisce
8597         .hack//g.u. vol.2//reminisce (jp sales)
8293                   .hack//g.u. vol.3//redemption
1561                         .hack//infection part 1
9063                                     .hack//link
2990                          .hack//mutation part 2
4265                          .hack//outbreak part 3
7992     .hack//quarantine part 4: the final chapter
14329              .hack: sekai no mukou ni + versus
3757                                      007 racing
4475                          007: quantum of solace
14658                         007: quantum of solace
1285                          007: quantum of solace
9507                          007: quantum of solace
1785                          007: quantum of solace
3120                          007: quantum of solace
2243                    007: the world is not 

Se oberva que la fila 7087 y 8597 parecen ser el mismo videojuego salvo por `'(jp sales)'`. Se imprimiran las dos filas para investigación.

In [20]:
for number in [7087, 8597]:
    print(df.loc[number])

name               .hack//g.u. vol.2//reminisce
platform                                    ps2
year_of_release                          2006.0
genre                              role-playing
na_sales                                   0.11
eu_sales                                   0.09
jp_sales                                    0.0
other_sales                                0.03
critic_score                                NaN
user_score                                  NaN
rating                                      NaN
Name: 7087, dtype: object
name               .hack//g.u. vol.2//reminisce (jp sales)
platform                                               ps2
year_of_release                                     2006.0
genre                                         role-playing
na_sales                                               0.0
eu_sales                                               0.0
jp_sales                                              0.16
other_sales                      

Los datos que no son sobre las ventas coinciden y en las ventas la diferencia es que el videojuego etiquetado con `'(jp sales)'` registra únicamente las ventas de Japón.

In [21]:
name_sales = df[df['name'].str.contains('sales')] #filtra la tabla solamente con las filas que contienen sales en la columna name
print(name_sales['name'].sort_values().unique())

['.hack//g.u. vol.2//reminisce (jp sales)' 'ape escape academy (jp sales)'
 'atelier iris 3: grand phantasm (jp sales)'
 'atelier iris 3: grand phantasm (us sales)'
 'battlefield 2: modern combat(jp sales)' 'bleach: dark souls (jp sales)'
 'bomberman (jp sales)' 'brave story: new traveler (us sales)'
 'brave story: new traveller (jp sales)'
 'chou-kuukan night pro yakyuu king (weekly jp sales)'
 'chou-kuukan night pro yakyuu king(higher jp sales)'
 'dance dance revolution: mario mix (jp sales)' 'draglade (jp sales)'
 'dragon ball z: budokai tenkaichi 2 (jp sales)'
 'dragon ball z: harukanaru densetsu (jp sales)'
 'dragon ball: origins (jp & incomplete us sales)'
 'dynasty warriors 6 (jp sales)' 'dynasty warriors vol. 2 (jp sales)'
 'earth defense force 2017 (old jp sales)' 'enchanted arms (jp sales)'
 'eternal poison (jp sales)'
 'fatal fury: battle archives volume 1 (jp sales)'
 'fatal fury: battle archives volume 2 (jp sales)'
 'fifa soccer 2003 (weekly jp sales)' 'fishing master (jp

Se oberva que varios títulos contienen entre paréntesis las ventas de alguna o más regiones pero también, existen 3 que contienen más información:
1. `'lunar 2: eternal blue(sales, but wrong system)'`.
2. `'pachi-slot teiou: golgo 13 las vegas (jp sales, but wrong system)'`.
3. `'r-type command (correct us sales)'`.

Se imprimirán las filas que contienen `'lunar 2: eternal blue'` para investigación.

In [22]:
print(df[df['name'].str.contains('lunar 2: eternal blue')])

                                                name platform  \
5331                  lunar 2: eternal blue complete       ps   
9000  lunar 2: eternal blue(sales, but wrong system)      gen   
9961                           lunar 2: eternal blue      sat   

      year_of_release         genre  na_sales  eu_sales  jp_sales  \
5331           1999.0  role-playing      0.19      0.13      0.00   
9000           1994.0  role-playing      0.00      0.00      0.14   
9961           1998.0  role-playing      0.00      0.00      0.12   

      other_sales  critic_score  user_score rating  
5331         0.02          86.0         8.9      t  
9000         0.00           NaN         NaN    NaN  
9961         0.00           NaN         NaN    NaN  


Son tres filas, con años distintos, esto tiene sentido porque fue lanzado para diferentes consolas. Buscando información, el videojuego del año 1994, fue lanzado para Sega CD (scd) y no para Sega Genesis (gen).

In [23]:
df.loc[9000, 'platform'] = 'scd' #corregimos la consola
print(df.loc[9000])

name               lunar 2: eternal blue(sales, but wrong system)
platform                                                      scd
year_of_release                                            1994.0
genre                                                role-playing
na_sales                                                      0.0
eu_sales                                                      0.0
jp_sales                                                     0.14
other_sales                                                   0.0
critic_score                                                  NaN
user_score                                                    NaN
rating                                                        NaN
Name: 9000, dtype: object


Ahora se imprimirán las filas que contienen `'pachi-slot teiou: golgo 13 las vegas'` para investigación.

In [24]:
print(df[df['name'].str.contains('pachi-slot teiou: golgo 13 las vegas')])

                                                    name platform  \
16153  pachi-slot teiou: golgo 13 las vegas (jp sales...      ps2   

       year_of_release genre  na_sales  eu_sales  jp_sales  other_sales  \
16153           2002.0  misc       0.0       0.0      0.01          0.0   

       critic_score  user_score rating  
16153           NaN         NaN    NaN  


Es una única fila y según la información buscada este videojuego fue lanzado para la consola PlayStation (ps).

In [25]:
df.loc[16153, 'platform'] = 'ps' #corregimos plataforma
print(df.loc[16153])

name               pachi-slot teiou: golgo 13 las vegas (jp sales...
platform                                                          ps
year_of_release                                               2002.0
genre                                                           misc
na_sales                                                         0.0
eu_sales                                                         0.0
jp_sales                                                        0.01
other_sales                                                      0.0
critic_score                                                     NaN
user_score                                                       NaN
rating                                                           NaN
Name: 16153, dtype: object


Se buscarán las filas que contienen `'r-type command'` para información.

In [26]:
print(df[df['name'].str.contains('r-type command')])

                                    name platform  year_of_release     genre  \
11956                     r-type command      psp           2007.0  strategy   
13946  r-type command (correct us sales)      psp           2007.0  strategy   

       na_sales  eu_sales  jp_sales  other_sales  critic_score  user_score  \
11956      0.05       0.0      0.01         0.01          69.0         6.5   
13946      0.04       0.0      0.00         0.00           NaN         NaN   

      rating  
11956   e10+  
13946    NaN  


Únicamente son dos filas, el mensaje entre paréntesis indica que es el dato correcto de las ventas en Norte América.

In [27]:
df.loc[11956, 'na_sales'] = 0.04 #corregimos na_sales
print(df.loc[11956])

name               r-type command
platform                      psp
year_of_release            2007.0
genre                    strategy
na_sales                     0.04
eu_sales                      0.0
jp_sales                     0.01
other_sales                  0.01
critic_score                 69.0
user_score                    6.5
rating                       e10+
Name: 11956, dtype: object


In [28]:
df = df.drop(index=13946) #eliminamos fila
print(df[df['name'].str.contains('r-type command')])

                 name platform  year_of_release     genre  na_sales  eu_sales  \
11956  r-type command      psp           2007.0  strategy      0.04       0.0   

       jp_sales  other_sales  critic_score  user_score rating  
11956      0.01         0.01          69.0         6.5   e10+  


Se eliminarán los mensajes entre paréntesis que se encontraron de los títulos de los videojuegos.

In [29]:
sales_list = ['(jp sales)', '(us sales)', '(weekly jp sales)', '(higher jp sales)', '(jp & incomplete us sales)',
              '(old jp sales)', '(others sales)', '(american sales)', '(all region sales)', '(jp weekly sales)',
              '(america weekly sales)', '(weekly american sales)', '(us weekly sales)', '(old us sales)',
              '(old all region sales)', '(us & others sales)', '(jp & others sales)', '(japan sales)',
              '(american and others sales)', '(sales, but wrong system)', '(jp sales, but wrong system)', '(all regions sales)']

In [30]:
def delete_word(string, word): #función que evalua si el segundo argumento está en el primero
    if word in string:
        string = string.replace(word, '') #remueve el segundo argumento del primero
        string = string.strip()
        return string
    else:
        return string
    
def name_replace(string): #función que elimina el elemento de la lista del título (en caso de contenerlo)
    for sale in sales_list: #recorre los elementos de la lista
        string = delete_word(string, sale) #evalua la primera función para el elemento correspondiente del bucle
    return string

In [31]:
df_test = pd.DataFrame() #DataFrame de prueba
df_test['name'] = ['game 1 (jp sales)', 'g2 us sales', 'g3(jp weekly sales)', 'g4']
df_test['year'] = [2004, 2005, 2006, 2007]
print(df_test)
print()
df_test['name'] = df_test['name'].apply(name_replace) #se aplica la función al DataFrame de prueba
print(df_test)

                  name  year
0    game 1 (jp sales)  2004
1          g2 us sales  2005
2  g3(jp weekly sales)  2006
3                   g4  2007

          name  year
0       game 1  2004
1  g2 us sales  2005
2           g3  2006
3           g4  2007


In [32]:
df['name'] = df['name'].apply(name_replace) #se aplica la función a df
df['name'] = df['name'].str.strip() #se eliminan espacios en blanco al inicio o final del título (de existir)
print(df[df['name'].str.contains('sales')]) #se verifica que no existan más títulos con la palabra sales

Empty DataFrame
Columns: [name, platform, year_of_release, genre, na_sales, eu_sales, jp_sales, other_sales, critic_score, user_score, rating]
Index: []


Se han eliminado los elementos de la lista que se creó. Se imprimirá nuevamente la longitud de los valores únicos en la colunmna `'name'`.

In [33]:
print(len(df['name'].sort_values().unique()))

11418


Esto indica la posibilidad que existan títulos con mensajes entre paréntesis, los cuales pueden contener información sobre los datos de las otras columnas.

In [34]:
name_extra = df[df['name'].str.endswith(')')] #filtramos la tabla con las filas cuyo título termina con )
print(name_extra['name'].sort_values().unique())

['attack on titan (koei)' 'blood: the last vampire (joukan)'
 "cabela's big game hunter (2008)" "cabela's outdoor adventures (2006)"
 "cabela's outdoor adventures (2009)" 'dance dance revolution (japan)'
 'dance dance revolution (north america)' 'doom (2016)'
 'f1 2016 (codemasters)' 'fuse (insomniac)' 'gadget racers (pal)'
 'gallop racer (jp)' 'game of thrones (telltale)' 'ghostbusters (2016)'
 'ghostbusters: the video game (ds version)' 'godzilla (2014)'
 'godzilla (2015)' 'goldeneye 007 (2010)' 'gran turismo (psp)'
 'hajime no ippo: the fighting! (2014)' 'hitman (2016)'
 'jampack spring 2004 (rp-t)' 'jampack volume 13 (rp-m)'
 'jampack volume 13 (rp-t)' 'jampack volume 14 (rp-t)'
 'jampack winter 2003 (rp-m)' 'jampack winter 2003 (rp-t)'
 'jampack: summer 2003 (rp-m)' 'jampack: summer 2003 (rp-t)'
 'killer instinct (2013)' 'kimikiss (ebkore+)' "king's field (japan)"
 'mad max (2015)' 'motor toon grand prix (japan)' 'need for speed (2015)'
 "nobunaga's ambition (3ds)" 'prince of pers

Algunos contienen las consolas de lanzamiento nuevamente pero ahora entre paréntesis. Se eliminará esta información de los títulos.

In [35]:
name_platform =['(ds version)', '(ds versions)', '(psp)', '(3ds)', '(cd)', '(ps1 & psp versions)', '(console version)',
                '(ds & mobile versions)', '(wii & ps2 version)', '(xbox 360, ps3, & pc versions)',
                '(xbox 360, ps2, ps3, wii & pc versions)', '(gbc)']

In [36]:
def name_plat(string): #función que elimina el elemento de la lista del título (en caso de contenerlo)
    for plat in name_platform: #recorre los elementos de la lista
        string = delete_word(string, plat) #evalua la primera función para el elemento correspondiente del bucle
    return string

In [37]:
df_test = pd.DataFrame() #DataFrame de prueba
df_test['name'] = ['game 1 (ds version)', 'g2 psp', 'g3(3ds)', 'g4']
df_test['year'] = [2004, 2005, 2006, 2007]
print(df_test)
print()
df_test['name'] = df_test['name'].apply(name_plat) #se aplica la función al DataFrame de prueba
print(df_test)

                  name  year
0  game 1 (ds version)  2004
1               g2 psp  2005
2              g3(3ds)  2006
3                   g4  2007

     name  year
0  game 1  2004
1  g2 psp  2005
2      g3  2006
3      g4  2007


In [38]:
df['name'] = df['name'].apply(name_plat) #aplicamos la función a df
print(len(df['name'].sort_values().unique()))

11406


In [39]:
name_extra = df[df['name'].str.endswith(')')] #filtramos nuevamente la tabla con las filas cuyo título termina con )
print(name_extra['name'].sort_values().unique())

['attack on titan (koei)' 'blood: the last vampire (joukan)'
 "cabela's big game hunter (2008)" "cabela's outdoor adventures (2006)"
 "cabela's outdoor adventures (2009)" 'dance dance revolution (japan)'
 'dance dance revolution (north america)' 'doom (2016)'
 'f1 2016 (codemasters)' 'fuse (insomniac)' 'gadget racers (pal)'
 'gallop racer (jp)' 'game of thrones (telltale)' 'ghostbusters (2016)'
 'godzilla (2014)' 'godzilla (2015)' 'goldeneye 007 (2010)'
 'hajime no ippo: the fighting! (2014)' 'hitman (2016)'
 'jampack spring 2004 (rp-t)' 'jampack volume 13 (rp-m)'
 'jampack volume 13 (rp-t)' 'jampack volume 14 (rp-t)'
 'jampack winter 2003 (rp-m)' 'jampack winter 2003 (rp-t)'
 'jampack: summer 2003 (rp-m)' 'jampack: summer 2003 (rp-t)'
 'killer instinct (2013)' 'kimikiss (ebkore+)' "king's field (japan)"
 'mad max (2015)' 'motor toon grand prix (japan)' 'need for speed (2015)'
 'prince of persia (2008)' 'rain (2013)' 'ratchet & clank (2016)'
 'resident evil (remake)' 'simcity (2013)' '

Algunos títulos contienen un año o bien una región. Sin embargo, parece que esto indica un nuevo juego dentro de la misma serie. Se decide dejar esta etiqueta. Los títulos que comienzan con `'jampack'` parece que contienen su clasificación (ESRB), se imprimirán dichas filas para su investigación.

In [40]:
print(df[df['name'].str.startswith('jampack')])

                              name platform  year_of_release genre  na_sales  \
1061            jampack winter '99       ps           1999.0  misc      0.94   
1171             jampack summer 2k       ps           2000.0  misc      0.88   
1813            jampack winter '98       ps           1998.0  misc      0.63   
2075           jampack winter 2000       ps           2000.0  misc      0.55   
2140            jampack summer '99       ps           1999.0  misc      0.54   
2746           jampack summer 2001      ps2           2001.0  misc      0.36   
2843           jampack winter 2001      ps2           2001.0  misc      0.35   
3163      jampack volume 13 (rp-t)      ps2           2005.0  misc      0.31   
3386             jampack fall 2001       ps           2001.0  misc      0.33   
3523           jampack summer 2002      ps2           2002.0  misc      0.28   
3750             jampack volume 11      ps2           2004.0  misc      0.26   
3768                jampack vol. 2      

Existen títulos cuya única diferencia en la información entre paréntesis, sin embargo, tienen ventas distintas por lo que en esencia son dos versiones distintas de la serie. Se decide continuar con estas etiquetas.

Se creará una función para reemplazar el valor ausente en la clasificación de estas filas por '`rp'`.

In [43]:
def replace_rating(row):
    name = str(row['name'])
    rating = row['rating']
    if re.search('rp-', name):
        rating = 'rp'
        return rating
    else:
        return rating

In [44]:
df_test = pd.DataFrame()
df_test['name'] = ['jampack winter 2003 (rp-m)', 'fifa', 'jampack winter 2003 (rp-t)', 'fifa 20']
df_test['rating'] = [float('nan'), float('nan'), float('nan'), 'e']
print(df_test)
print()
df_test['rating'] = df_test.apply(replace_rating, axis=1)
print(df_test)

                         name rating
0  jampack winter 2003 (rp-m)    NaN
1                        fifa    NaN
2  jampack winter 2003 (rp-t)    NaN
3                     fifa 20      e

                         name rating
0  jampack winter 2003 (rp-m)     rp
1                        fifa    NaN
2  jampack winter 2003 (rp-t)     rp
3                     fifa 20      e


In [45]:
df['rating'] = df.apply(replace_rating, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16712 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16712 non-null  object 
 1   platform         16712 non-null  object 
 2   year_of_release  16443 non-null  float64
 3   genre            16712 non-null  object 
 4   na_sales         16712 non-null  float64
 5   eu_sales         16712 non-null  float64
 6   jp_sales         16712 non-null  float64
 7   other_sales      16712 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9957 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB


In [46]:
print(len(df['name'].unique()))

11406


Debido a que es una cantidad muy grande de valores únicos no se pueden revisar uno por uno en busca de ducplicados implícitos (por el tiempo). Se decide tratarlos como 11,406 juegos distintos.

**Conclusiones**

En esta sección se trataron las columnas con datos tipo object, en particular para buscar duplicados implícitos. Las columnas `'platform'` y `'genre'` no contenían duplicados implícitos. Se elimnaron las filas con valores ausentes en la columna `'genre'` (eran las mismas con valores ausentes para la columna `'name'`). La columna `'rating'` contenía las clasificaciones `'e'` y `'k-a'` que representan la misma información (apto pra todo público), por lo que se reemplazó `'k-a'` por `'e'`. La columna `'user_score'` contenía el valor `'tbd'` lo que significaba que el videojuego no había sido puntuado por lo que se reemplazaron estos datos por NaN (valor ausente) y se cambiaron los datos de esta columna a tipo float.

En la siguiente sección se tratarán los valores ausentes.

[Volver a Contenidos](#back)

### 1. 2. Valores ausentes <a id='data_review_missing'></a>

Se imprimirá la cantidad de valores ausentes de cada columna para proceder a tratarlos.

In [47]:
print(df.isna().sum())

name                  0
platform              0
year_of_release     269
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8575
user_score         9122
rating             6755
dtype: int64


#### 1.2.1 Columna `'year_of_release'` <a id='data_review_missing_year'></a>

Los valores ausentes en la columna `'year_of_release'` pueden deberse a un error en la recopilación de los datos. Algunos juegos se lanzan para diversas plataformas o consolas, de manera que se pueden reemplazar estos datos por el año de lanzamiento del juego para otra consola. Para esto, se definirá una función que analice el título del videojuego y asigne el año de lanzamiento para otra consola en caso de existir.

In [48]:
df_year_nonan = df.dropna(subset=['name']) #tabla sin valores ausentes en la columna del año de lanzamiento
year_nonan = df_year_nonan.pivot_table(index='name', values='year_of_release')#promedio de años de un juego (diferentes consolas)
year_nonan.info()
print()
print(year_nonan.head(10))

<class 'pandas.core.frame.DataFrame'>
Index: 11278 entries, '98 koshien to ¡shin chan flipa en colores!
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year_of_release  11278 non-null  float64
dtypes: float64(1)
memory usage: 176.2+ KB

                                             year_of_release
name                                                        
'98 koshien                                           1998.0
.hack//g.u. vol.1//rebirth                            2006.0
.hack//g.u. vol.2//reminisce                          2006.0
.hack//g.u. vol.3//redemption                         2007.0
.hack//infection part 1                               2002.0
.hack//link                                           2010.0
.hack//mutation part 2                                2002.0
.hack//outbreak part 3                                2002.0
.hack//quarantine part 4: the final chapter           2003.0
.hack: seka

In [50]:
def year_release(row):
    name = row['name']
    year = row['year_of_release']
    if pd.isna(year):#evalua si es valor ausente
        try:#si esxiste el juego en la tabla year_nonan
            year = year_nonan.loc[name]
        except:
            year = year
        return year
    else:
        return year

In [51]:
df_test = pd.DataFrame()#DataFrame de prueba
df_test['name'] = ["'98 koshien", '.hack//g.u. vol.1//rebirth', 'Fifa', 'hola mundo']
df_test['year_of_release'] = [float('nan'), float('nan'), 2014, float('nan')]
print(df_test)
print()
df_test['year_of_release'] = df_test.apply(year_release, axis=1)
print(df_test)

                         name  year_of_release
0                 '98 koshien              NaN
1  .hack//g.u. vol.1//rebirth              NaN
2                        Fifa           2014.0
3                  hola mundo              NaN

                         name  year_of_release
0                 '98 koshien           1998.0
1  .hack//g.u. vol.1//rebirth           2006.0
2                        Fifa           2014.0
3                  hola mundo              NaN


In [52]:
df['year_of_release'] = df.apply(year_release, axis=1)#se aplica la función a df
print(df.isna().sum())

name                  0
platform              0
year_of_release     141
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8575
user_score         9122
rating             6755
dtype: int64


Aún existen valores ausentes, sin embargo estas filas representan menos del 1% de toda la tabla. Se investigará la media y mediana de los años de lanzamientos de la tabla sin valores ausentes (en la columna del año de lanzamiento) pero agrupada por consolas pues pueden existir juegos para consolas muy antiguas. Se decidirá reemplazar por alguna de las dos.

In [56]:
platform_year = df_year_nonan.pivot_table(index='platform', values='year_of_release', aggfunc=['mean', 'median'])
platform_year.columns = ['mean', 'median']
print(platform_year.head())

                 mean  median
platform                     
2600      1982.137931  1982.0
3do       1994.666667  1995.0
3ds       2013.126953  2013.0
dc        1999.942308  2000.0
ds        2008.185290  2008.0


Estos dos valores son muy similares, de hecho, la medianda coincide con el redondeo de la media. Se decide utilizar la mediana para reemplazar los valores ausentes.

In [63]:
def year_release2(row):
    platform = str(row['platform'])
    year = float(row['year_of_release'])
    if pd.isna(year):#evalua si es valor ausente
        try:#si esxiste el juego en la tabla year_nonan
            year = platform_year.loc[platform, 'median']
        except:
            year = year
        return year
    else:
        return year

In [64]:
df_test = pd.DataFrame()#DataFrame de prueba
df_test['platform'] = ['2600', '3do', 'xb', 'xbone']
df_test['year_of_release'] = [float('nan'), float('nan'), 2014, float('nan')]
print(df_test)
print()
df_test['year_of_release'] = df_test.apply(year_release2, axis=1)
print(df_test)

  platform  year_of_release
0     2600              NaN
1      3do              NaN
2       xb           2014.0
3    xbone              NaN

  platform  year_of_release
0     2600           1982.0
1      3do           1995.0
2       xb           2014.0
3    xbone              NaN


In [65]:
df['year_of_release'] = df.apply(year_release2, axis=1)
print(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8575
user_score         9122
rating             6755
dtype: int64


In [67]:
df['year_of_release'] = df['year_of_release'].astype('int')#convertimos los datos de esta columna a tipo int
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16712 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16712 non-null  object 
 1   platform         16712 non-null  object 
 2   year_of_release  16712 non-null  int64  
 3   genre            16712 non-null  object 
 4   na_sales         16712 non-null  float64
 5   eu_sales         16712 non-null  float64
 6   jp_sales         16712 non-null  float64
 7   other_sales      16712 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9957 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


Ya no existen valores ausentes en esta columna. Además, se redondearon los valores pues el año de lanzamiento debe ser un valor entero.

[Volver a Contenidos](#back)

#### 1.2.2 Columna `'critic_score'` <a id='data_review_missing_criticscore'></a>

Para tratar los valores ausentes de la columna `'critic_score'` primero se creará una tabla sin éstos y se creará una nueva con el promedio de las puntuaciones por título (la puntuación puede variar dependiendo de la consola).

In [68]:
df_critic_nonan = df.dropna(subset='critic_score')
critic_nonan = df_critic_nonan.pivot_table(index='name', values='critic_score')
critic_nonan.info()
print()
print(critic_nonan.head(10))

<class 'pandas.core.frame.DataFrame'>
Index: 5077 entries, .hack//infection part 1 to zumba fitness: world party
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   critic_score  5077 non-null   float64
dtypes: float64(1)
memory usage: 79.3+ KB

                               critic_score
name                                       
.hack//infection part 1                75.0
.hack//mutation part 2                 76.0
.hack//outbreak part 3                 70.0
007 racing                             51.0
007: quantum of solace                 63.8
007: the world is not enough           61.0
1 vs. 100                              38.0
100 classic books                      70.0
101-in-1 explosive megamix             46.0
101-in-1 sports party megamix          41.0


Se creará una función que asigne la puntuación a un título con valor ausente si es que se encuentra dentro de los títulos de la tabla anterior.

In [69]:
def critic_score(row):
    name = row['name']
    critic = row['critic_score']
    if pd.isna(critic):
        try:
            critic = float(critic_nonan.loc[name])
        except:
            critic = critic
        return critic
    else:
        return critic

Se creará una tabla de prueba para verificar si la función es correcta.

In [70]:
df_test = pd.DataFrame()
df_test['name'] = ['.hack//infection part 1', '007 racing', 'XYZ', 'Fifa']
df_test['critic_score'] = [float('nan'), float('nan'), float('nan'), 23]
print(df_test)
print()
df_test['critic_score'] = df_test.apply(critic_score, axis=1)
print(df_test)

                      name  critic_score
0  .hack//infection part 1           NaN
1               007 racing           NaN
2                      XYZ           NaN
3                     Fifa          23.0

                      name  critic_score
0  .hack//infection part 1          75.0
1               007 racing          51.0
2                      XYZ           NaN
3                     Fifa          23.0


La función es correcta. Se aplicará a `df`.

In [71]:
df['critic_score'] = df.apply(critic_score, axis=1)
print(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       7519
user_score         9122
rating             6755
dtype: int64


Aún existe una cantidad considerable de valores ausentes en esta columna. Como no es la variable principal de interés, se reemplazarán estos valores ausentes con la media de las puntuaciones dependiendo de la consola y el género. Para esto, se creará una tabla dinámica donde las filas serán los tipos de consolas y las columnas serán los géneros.

In [72]:
critic_pivot = df_critic_nonan.pivot_table(index='platform', columns='genre', values='critic_score')
print(critic_pivot)

genre        action  adventure   fighting       misc   platform     puzzle  \
platform                                                                     
3ds       63.118644  67.500000  68.857143  69.100000  72.444444  67.100000   
dc              NaN  88.000000  89.000000        NaN  89.000000        NaN   
ds        58.309353  61.372881  63.133333  63.322581  67.509091  66.722772   
gba       62.152381  65.928571  67.250000  64.518519  68.074074  70.318182   
gc        67.747126  64.571429  69.962963  63.166667  63.813559  66.600000   
pc        74.304965  71.875000  80.000000  73.200000  74.800000  81.500000   
ps        67.117647  58.000000  67.625000  73.833333  76.000000  81.000000   
ps2       66.725100  62.348837  70.063291  67.888889  66.410959  72.636364   
ps3       69.024896  68.300000  73.298246  71.960000  68.565217  75.000000   
ps4       72.107143  73.428571  72.636364  70.555556  71.727273  88.000000   
psp       63.898876  63.461538  67.062500  66.034483  69.541667 

In [73]:
def critic_mean(row):
    platform = row['platform']
    genre = row['genre']
    critic = row['critic_score']
    if pd.isna(critic):
        try:
            critic = float(critic_pivot.loc[platform, genre])
        except:
            critic = critic
    else:
        return critic
    return critic

In [74]:
df_test = pd.DataFrame()
df_test['platform'] = ['3DS', 'DC', 'DS', 'NES']
df_test['genre'] = ['Action', 'Misc', 'Adventure', 'Fighting']
df_test['critic_score'] = [float('nan'), float('nan'), 23.5, float('nan')]
print(df_test)
print()
df_test['critic_score'] = df_test.apply(critic_mean, axis=1)
print(df_test)

  platform      genre  critic_score
0      3DS     Action           NaN
1       DC       Misc           NaN
2       DS  Adventure          23.5
3      NES   Fighting           NaN

  platform      genre  critic_score
0      3DS     Action           NaN
1       DC       Misc           NaN
2       DS  Adventure          23.5
3      NES   Fighting           NaN


La función es correcta, se aplicará a `df`.

In [75]:
df['critic_score'] = df.apply(critic_mean, axis=1)
print(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       1070
user_score         9122
rating             6755
dtype: int64


Aún quedan valores ausentes, por lo que se reemplazará con la media o mediana de las puntuaciones por género.

In [76]:
critic_pivot2 = df_critic_nonan.pivot_table(index='genre', values='critic_score', aggfunc=['mean', 'median'])
critic_pivot2.columns = ['mean', 'median']
print(critic_pivot2)

                   mean  median
genre                          
action        66.629101    68.0
adventure     65.331269    66.0
fighting      69.217604    72.0
misc          66.619503    69.0
platform      68.058350    69.0
puzzle        67.424107    70.0
racing        67.963612    69.0
role-playing  72.652646    74.0
shooter       70.181144    73.0
simulation    68.619318    70.0
sports        71.968174    75.0
strategy      72.086093    73.0


Como las medias y medianas son distintas, se utilizará la mediana para reemplazar los valores ausentes que aún quedan.

In [77]:
def critic(row):
    genre = str(row['genre'])
    critic = float(row['critic_score'])
    if pd.isna(critic):#evalua si es valor ausente
        critic = critic_pivot2.loc[genre, 'median']
        return critic
    else:
        return critic

In [78]:
df_test = pd.DataFrame()
df_test['genre'] = ['action', 'adventure', 'fighting', 'misc']
df_test['critic_score'] = [float('nan'), float('nan'), 23.5, float('nan')]
print(df_test)
print()
df_test['critic_score'] = df_test.apply(critic, axis=1)
print(df_test)

       genre  critic_score
0     action           NaN
1  adventure           NaN
2   fighting          23.5
3       misc           NaN

       genre  critic_score
0     action          68.0
1  adventure          66.0
2   fighting          23.5
3       misc          69.0


In [79]:
df['critic_score'] = df.apply(critic, axis=1)
print(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score          0
user_score         9122
rating             6755
dtype: int64


Ya no existen valores ausentes en esta columna.

[Volver a Contenidos](#back)

#### 1.2.3 Columna `'user_score'` <a id='data_review_missing_userscore'></a>

Para tratar los valores ausentes de la columna `'user_score'` primero se imprimirá las estadísticas descriptivas de esta columna.

In [80]:
print(df['user_score'].describe())

count    7590.000000
mean        7.125046
std         1.500006
min         0.000000
25%         6.400000
50%         7.500000
75%         8.200000
max         9.700000
Name: user_score, dtype: float64


Como los valores están dentro de un rango específico se reemplazarán los valores ausentes por la media.

In [81]:
user_mean = df['user_score'].mean()
df['user_score'] = df['user_score'].fillna(user_mean)
print(df.isna().sum())

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score          0
user_score            0
rating             6755
dtype: int64


Ya no existen valores ausentes en esta columna.

[Volver a Contenidos](#back)

#### 1.2.4 Columna `'rating'` <a id='data_review_missing_rating'></a>

Los datos de la columna `'rating'` son variables categóricas, por esta razón, los valores ausentes se reemplazarán con el valor más frecuente por género.

In [87]:
df_rating_nonan = df.dropna(subset='rating')

genre_list = ['action', 'adventure', 'fighting', 'misc', 'platform', 'puzzle', 'racing', 'role-playing',
              'shooter', 'simulation', 'sports', 'strategy']

rating = {} #diccionario de dataframe por género
for genre in genre_list:
    rating[genre] = df_rating_nonan[df_rating_nonan['genre'] == genre]
    
print(rating.keys())

dict_keys(['action', 'adventure', 'fighting', 'misc', 'platform', 'puzzle', 'racing', 'role-playing', 'shooter', 'simulation', 'sports', 'strategy'])


In [90]:
def mode_rating(data):
    rat = data['rating']
    frequency = {}#diccionario rating-frecuencia
    for element in rat:
        frequency[element] = frequency.get(element, 0) + 1
    most_frequent = max(frequency.values())#frecuencia maxima
    for key, value in frequency.items():#devuelve el rating
        if value == most_frequent:
            mode = key
    return mode

In [91]:
df_test = pd.DataFrame()#DataFrame de prueba
df_test['genre'] = ['action', 'adventure', 'fighting', 'misc']
df_test['rating'] = ['m', 't', 'm', 'rp']
print(df_test)
print()
mode_df_test = mode_rating(df_test)
print(mode_df_test)

       genre rating
0     action      m
1  adventure      t
2   fighting      m
3       misc     rp

m


In [92]:
modes = {}#diccionario género-moda
for genre in genre_list:
    modes[genre] = mode_rating(rating[genre])

print(modes)

{'action': 't', 'adventure': 'e', 'fighting': 't', 'misc': 'e', 'platform': 'e', 'puzzle': 'e', 'racing': 'e', 'role-playing': 't', 'shooter': 'm', 'simulation': 'e', 'sports': 'e', 'strategy': 't'}


In [93]:
def assign_rating(row):
    genre = str(row['genre'])
    rating = row['rating']
    if pd.isna(rating):
        rating = modes[genre]
        return rating
    else:
        return rating

In [95]:
df_test = pd.DataFrame()#DataFrame de prueba
df_test['genre'] = ['action', 'adventure', 'fighting', 'misc']
df_test['rating'] = [float('nan'), 't', float('nan'), 'rp']
print(df_test)
print()
df_test['rating'] = df_test.apply(assign_rating, axis=1)
print(df_test)

       genre rating
0     action    NaN
1  adventure      t
2   fighting    NaN
3       misc     rp

       genre rating
0     action      t
1  adventure      t
2   fighting      t
3       misc     rp


In [96]:
df['rating'] = df.apply(assign_rating, axis=1)#aplicamos la función a df
print(df.isna().sum())

name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64


Ya no existen valores ausentes en esta columna.

**Conclusiones**
Se reemplazaron los valores ausentes de las columnas `'year_of_release'`, `'critic_score'` y `'user_score'` con medias o medianas dependiendo de las coonsolas y los géneros de los videojuegos. Para la columna `'rating'` se utilizó la moda de las clasificaciones por género de los videojuegos. Esto fue así porque los datos son categóricos.

[Volver a Contenidos](#back)

### 1.3 Conclusiones <a id='data_review_conclusions'></a>

Se examinó el DataFrame, donde las columnas que no presentaron valores ausentes fuero `'platform'` y la de las ventas. Se cambiaron a minúsculas todas las letras de los títulos de las columnas, así como los valores de las columnas de tipo object. Se eliminaron las 2 filas que contenían los valores ausentes tanto de la columna `'name'` como de `'genre'`.

Se analizaron las columnas de tipo object en busqueda de duplicados implícitos. En la columna `'rating'` se unificaron dos valores que aportan la misma información: `'k-a'` y `'e'`. Se eliminaron algunos duplicados implícitos de la columna `'name'`, sin embargo tendría que revisarse con más detalle para asegurar que ya no existen tales duplicados.

Se convirtieron los datos de la columna `'year_of_release'` a tipo int. Se reemplazaron los valores ausentes de las columnas numéricas con la media o mediana dependiendo de consola y género, mientras que los de la columna con valores categóricos se reemplazaron con la moda de la clasificación por género.

Los datos son suficientes para comprobar hipótesis.

[Volver a Contenidos](#back)